Copyright (C) 2024 Max Wendler <max.wendler@gmail.com>

SPDX-License-Identifier: GPL-2.0-or-later

This program is free software; you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation; either version 2 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program; if not, write to the Free Software
Foundation, Inc., 59 Temple Place, Suite 330, Boston, MA  02111-1307  USA

Notebook for placing median constellation epoch in middle of simulation time under consideration of warmup and cooldown time

 - Only runs in devcontainer!

In [1]:
from tleparse import read
# paths of constellations/tle sets to analyze
tles_paths = ["/workspaces/ma-max-wendler/examples/space_veins/tles/avg_epoch_timestamps/undecomposed/iridiumNEXT_2023-10-22-21-39-19.txt",
                "/workspaces/ma-max-wendler/examples/space_veins/tles/avg_epoch_timestamps/undecomposed/starlinkShort_2023-10-22-21-18-24.txt",
                "/workspaces/ma-max-wendler/examples/space_veins/tles/avg_epoch_timestamps/undecomposed/oneweb_2023-10-22-22-28-26.txt",
                "/workspaces/ma-max-wendler/examples/space_veins/tles/avg_epoch_timestamps/satnogs_leo_2023-10-22-23-58-09.txt",
                "/workspaces/ma-max-wendler/examples/space_veins/tles/avg_epoch_timestamps/eccentric_2023-10-22-21-03-39.txt"]

In [2]:
from astropy.time import Time
import astropy.units as u
from statistics import median

In [ ]:
for path in tles_paths:
    with open(path, "r") as in_f:
        org_lines = in_f.readlines()
    
    tles = read(path)
    epochs_unix = []
    for tle in tles:
        epoch = tle.epoch
        epoch.format = 'unix'
        epochs_unix.append(epoch.value)
    
    median_epoch_unix = median(epochs_unix)
    median_epoch_time = Time(median_epoch_unix, format='unix' ,scale='utc')

    # + 5 seconds, half of warmup time - 5 to balance with cooldown
    half_day_before_median = median_epoch_time - ( 43205 << u.s)
    half_day_before_median.format = 'datetime'
    half_day_before_median_dt_string = half_day_before_median.value.strftime("%Y-%m-%d-%H-%M-%S")
    print(half_day_before_median_dt_string)

    output_modified_path = path.replace("/avg_epoch_timestamps","")
    constellation = path.split("/")[-1].split("_")[0]
    new_path = "/".join(output_modified_path.split("/")[:-1]) + "/" + constellation + "_" + half_day_before_median_dt_string + ".txt"
    with open(new_path, "w") as out_f:
        out_f.writelines(org_lines)